In [1]:
# import subprocess
import encode as ec
from models import speeds, model_paths, dorado
from IPython.display import Markdown, display
bash = lambda commands: display(Markdown("```bash\n" + ' && \n'.join(commands) + "\n```"))

data_dir = 'pruned_out'
# def run(command, **kwargs):
#   return subprocess.run(command, check=True, cwd=data_dir, **kwargs)

We prepare data by using Jasmine's script to obtain `data_nuc.txt` encoded into DNA nucleotides under the repository, then convert into `sample.fasta` under data directory

In [82]:
data_src_dir = 'dna_archival_storage/encoding_data'

ec.ascii_to_binary(f'{data_src_dir}/data.txt', f'{data_src_dir}/data_bin.txt')
ec.binary_to_nt(f'{data_src_dir}/data_bin.txt', f'{data_src_dir}/data_nuc.txt')

with open(f'{data_src_dir}/data_nuc.txt', "r") as infile:
  sequence = "".join(line.strip() for line in infile if line.strip())

with open(f'{data_dir}/sample.fasta', "w") as outfile:
  outfile.write(">a_tale_of_two_cities\n")
  outfile.write(sequence + "\n")
  # for i in range(0, len(sequence), 60):
  #   outfile.write(sequence[i:i+60] + "\n")

f'{data_dir}/sample.fasta'

'perfect_out/sample.fasta'

(Deprecated) ~~We use the `Seq2Squiggle` library to obtain the pod5 prediction in `sample.pod5`~~

In [73]:
# run([
#   "seq2squiggle", "predict", "sample.fasta",
#   "-o", "sample.pod5",
#   "--profile", "dna-r10-min",
#   "--read-input"
# ])
# bash(["seq2squiggle predict sample.fasta -o sample.pod5 --profile dna-r10-min --read-input"])

We use the `squigulator` library (as a sanity check) to output `sample.slow5`

In [3]:
bash(["squigulator sample.fasta -x dna-r10-prom -o sample.slow5 --full-contigs"])

```bash
squigulator sample.fasta -x dna-r10-prom -o sample.slow5 --full-contigs
```

Before converting to `sample.pod5`, use the below code to correctly replace the read_id of the `slow5` file with a uuid as per `blue-crab`'s requirement:

In [83]:
import uuid
import os

input_file = f"{data_dir}/sample.slow5"
temp_file = f"{data_dir}/sample.slow5.tmp"

with open(input_file, "r") as fin, open(temp_file, "w") as fout:
  for line in fin:
    if line.startswith("#") or line.startswith("@"):
      fout.write(line)
    else:
      parts = line.strip().split('\t')
      parts[0] = str(uuid.uuid4())
      fout.write('\t'.join(parts) + '\n')

os.replace(temp_file, input_file)  # Atomically replace original file

In [79]:
bash(["blue-crab s2p sample.slow5 -o sample.pod5"])

```bash
blue-crab s2p sample.slow5 -o sample.pod5
```

We use `dorado` to basecall into `sample.fastq` using the 3 models we have profiled

In [19]:
# def basecall(speed):
#   with open(f"{data_dir}/{speed}.fastq", "w") as out:
#     run(
#       [dorado, "basecaller", model_paths[speed], "sample.pod5", "--emit-fastq"],
#       stdout=out,
#     )

# basecall('fast')
# basecall('hac')
# basecall('sup')

bash([f'dorado basecaller ${speed.upper()} sample.pod5 --emit-fastq > {speed}.fastq' for speed in speeds])

```bash
dorado basecaller $FAST sample.pod5 --emit-fastq > fast.fastq && 
dorado basecaller $HAC sample.pod5 --emit-fastq > hac.fastq && 
dorado basecaller $SUP sample.pod5 --emit-fastq > sup.fastq
```

We can now use `dorado`'s aligner to align the `sample.fasta` and `.fastq` files

In [22]:
# for speed in speeds:
#   with open(f'{data_dir}/{speed}.bam', 'w') as out:
#     run(
#       [dorado, "aligner", "sample.fasta", f'{speed}.fastq'],
#       stdout=out
#     )

bash([f'dorado aligner sample.fasta {speed}.fastq > {speed}.bam' for speed in speeds])

```bash
dorado aligner sample.fasta fast.fastq > fast.bam && 
dorado aligner sample.fasta hac.fastq > hac.bam && 
dorado aligner sample.fasta sup.fastq > sup.bam
```

We then use `dorado`'s summary utility to obtain the basecalling accuracy measures

In [21]:
# for speed in speeds:
#   with open(f'{data_dir}/{speed}.tsv', 'w') as out:
#     run(
#       [dorado, "summary", f'{speed}.bam'],
#       stdout=out
#     )

bash([f'dorado summary {speed}.bam > {speed}.tsv' for speed in speeds])

```bash
dorado summary fast.bam > fast.tsv && 
dorado summary hac.bam > hac.tsv && 
dorado summary sup.bam > sup.tsv
```

We collect the summary results and put into a markdown table

In [5]:
from IPython.display import Markdown, display

def tsv_dict(speed):
  with open(f'{data_dir}/{speed}.tsv', 'r') as f:
    a = f.readlines()
  return dict(zip(a[0].split(),a[1].split()))

def make_table(data, markdown=True):
  # row_headers = sorted({key for inner in data.values() for key in inner})
  row_headers = [
    "alignment_accuracy", "alignment_identity", 
    "alignment_genome_start", "alignment_genome_end", "alignment_strand_start", "alignment_strand_end", "alignment_strand_coverage",
    "alignment_length", "alignment_num_aligned", "alignment_num_correct", "alignment_num_deletions", "alignment_num_insertions", "alignment_num_substitutions",
    ]

  # Build markdown table
  header_row = f"| Metric | {' | '.join(data.keys())} |"
  separator_row = f"|--------|{'|'.join(['--------'] * len(data))}|"
  data_rows = [
    f"| {key} | {' | '.join(str(data[name].get(key, '')) for name in data)} |"
    for key in row_headers
  ]

  table_md = "\n".join([header_row, separator_row] + data_rows)

  if markdown:
    display(Markdown(table_md))
  else:
    print(table_md)

In [10]:
data = {speed: tsv_dict(speed) for speed in speeds}
make_table(data)

| Metric | fast | hac | sup |
|--------|--------|--------|--------|
| alignment_accuracy | 0.833501 | 0.873873 | 0.910638 |
| alignment_identity | 0.924108 | 0.942866 | 0.95575 |
| alignment_genome_start | 7 | 7 | 8 |
| alignment_genome_end | 3106830 | 3106840 | 3106840 |
| alignment_strand_start | 0 | 0 | 0 |
| alignment_strand_end | 2894020 | 2961899 | 3063253 |
| alignment_strand_coverage | 0.999996 | 0.999998 | 0.999999 |
| alignment_length | 3155097 | 3149601 | 3159611 |
| alignment_num_aligned | 2845746 | 2919131 | 3010474 |
| alignment_num_correct | 2629776 | 2752350 | 2877261 |
| alignment_num_deletions | 261077 | 187702 | 96358 |
| alignment_num_insertions | 48274 | 42768 | 52779 |
| alignment_num_substitutions | 215970 | 166781 | 133213 |

We observe that the hac model is the most stable. We perform one-shot pruning (in `prune_new.ipynb`) with different sparsities and evaluate their performances with the following workflow

In [3]:
test_model_dir = '/vol/bitbucket/bl1821/pruned_models'
sparsities = [0.0, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4]
pruned_models = [f'sparse_{s}' for s in sparsities]

bash(
  [f'dorado basecaller {test_model_dir}/{model} sample.pod5 --emit-fastq > {model}.fastq' for model in pruned_models] +
  [f'dorado aligner sample.fasta {model}.fastq > {model}.bam' for model in pruned_models] + 
  [f'dorado summary {model}.bam > {model}.tsv' for model in pruned_models]
)

```bash
dorado basecaller /vol/bitbucket/bl1821/pruned_models/sparse_0.0 sample.pod5 --emit-fastq > sparse_0.0.fastq && 
dorado basecaller /vol/bitbucket/bl1821/pruned_models/sparse_0.01 sample.pod5 --emit-fastq > sparse_0.01.fastq && 
dorado basecaller /vol/bitbucket/bl1821/pruned_models/sparse_0.05 sample.pod5 --emit-fastq > sparse_0.05.fastq && 
dorado basecaller /vol/bitbucket/bl1821/pruned_models/sparse_0.1 sample.pod5 --emit-fastq > sparse_0.1.fastq && 
dorado basecaller /vol/bitbucket/bl1821/pruned_models/sparse_0.2 sample.pod5 --emit-fastq > sparse_0.2.fastq && 
dorado basecaller /vol/bitbucket/bl1821/pruned_models/sparse_0.3 sample.pod5 --emit-fastq > sparse_0.3.fastq && 
dorado basecaller /vol/bitbucket/bl1821/pruned_models/sparse_0.4 sample.pod5 --emit-fastq > sparse_0.4.fastq && 
dorado aligner sample.fasta sparse_0.0.fastq > sparse_0.0.bam && 
dorado aligner sample.fasta sparse_0.01.fastq > sparse_0.01.bam && 
dorado aligner sample.fasta sparse_0.05.fastq > sparse_0.05.bam && 
dorado aligner sample.fasta sparse_0.1.fastq > sparse_0.1.bam && 
dorado aligner sample.fasta sparse_0.2.fastq > sparse_0.2.bam && 
dorado aligner sample.fasta sparse_0.3.fastq > sparse_0.3.bam && 
dorado aligner sample.fasta sparse_0.4.fastq > sparse_0.4.bam && 
dorado summary sparse_0.0.bam > sparse_0.0.tsv && 
dorado summary sparse_0.01.bam > sparse_0.01.tsv && 
dorado summary sparse_0.05.bam > sparse_0.05.tsv && 
dorado summary sparse_0.1.bam > sparse_0.1.tsv && 
dorado summary sparse_0.2.bam > sparse_0.2.tsv && 
dorado summary sparse_0.3.bam > sparse_0.3.tsv && 
dorado summary sparse_0.4.bam > sparse_0.4.tsv
```

In [6]:
data = {model: tsv_dict(model) for model in pruned_models}
make_table(data)

| Metric | sparse_0.0 | sparse_0.01 | sparse_0.05 | sparse_0.1 | sparse_0.2 | sparse_0.3 | sparse_0.4 |
|--------|--------|--------|--------|--------|--------|--------|--------|
| alignment_accuracy | 0.910638 | 0.910607 | 0.91101 | 0.912798 | 0.91608 | 0.795533 | 0 |
| alignment_identity | 0.95575 | 0.955668 | 0.956055 | 0.956132 | 0.95669 | 0.880613 | 0 |
| alignment_genome_start | 8 | 8 | 8 | 8 | 8 | 448431 | -1 |
| alignment_genome_end | 3106840 | 3106840 | 3106840 | 3106840 | 3106840 | 2881340 | -1 |
| alignment_strand_start | 0 | 0 | 0 | 0 | 0 | 416594 | -1 |
| alignment_strand_end | 3063253 | 3063204 | 3062044 | 3062846 | 3044361 | 2674383 | -1 |
| alignment_strand_coverage | 0.999999 | 0.999999 | 0.999999 | 0.999999 | 0.999999 | 0.78278 | 0 |
| alignment_length | 3159611 | 3159506 | 3158854 | 3156366 | 3142288 | 2464397 | 0 |
| alignment_num_aligned | 3010474 | 3010530 | 3010022 | 3013312 | 3008905 | 2226301 | 0 |
| alignment_num_correct | 2877261 | 2877067 | 2877748 | 2881124 | 2878588 | 1960509 | 0 |
| alignment_num_deletions | 96358 | 96302 | 96810 | 93520 | 97927 | 206608 | 0 |
| alignment_num_insertions | 52779 | 52674 | 52022 | 49534 | 35456 | 31488 | 0 |
| alignment_num_substitutions | 133213 | 133463 | 132274 | 132188 | 130317 | 265792 | 0 |

We then perform iterative pruning

In [8]:
iter_pruned_models = [f'step_{i}' for i in range(0,4)]

bash(
  [f'dorado basecaller {test_model_dir}/{model} sample.pod5 --emit-fastq > {model}.fastq' for model in iter_pruned_models] +
  [f'dorado aligner sample.fasta {model}.fastq > {model}.bam' for model in iter_pruned_models] +
  [f'dorado summary {model}.bam > {model}.tsv' for model in iter_pruned_models]
)

```bash
dorado basecaller /vol/bitbucket/bl1821/pruned_models/step_0 sample.pod5 --emit-fastq > step_0.fastq && 
dorado basecaller /vol/bitbucket/bl1821/pruned_models/step_1 sample.pod5 --emit-fastq > step_1.fastq && 
dorado basecaller /vol/bitbucket/bl1821/pruned_models/step_2 sample.pod5 --emit-fastq > step_2.fastq && 
dorado basecaller /vol/bitbucket/bl1821/pruned_models/step_3 sample.pod5 --emit-fastq > step_3.fastq && 
dorado aligner sample.fasta step_0.fastq > step_0.bam && 
dorado aligner sample.fasta step_1.fastq > step_1.bam && 
dorado aligner sample.fasta step_2.fastq > step_2.bam && 
dorado aligner sample.fasta step_3.fastq > step_3.bam && 
dorado summary step_0.bam > step_0.tsv && 
dorado summary step_1.bam > step_1.tsv && 
dorado summary step_2.bam > step_2.tsv && 
dorado summary step_3.bam > step_3.tsv
```

In [9]:
data = {model: tsv_dict(model) for model in iter_pruned_models}
make_table(data)

| Metric | step_0 | step_1 | step_2 | step_3 |
|--------|--------|--------|--------|--------|
| alignment_accuracy | 0.910627 | 0.91277 | 0.91277 | 0.91277 |
| alignment_identity | 0.955717 | 0.956137 | 0.956137 | 0.956137 |
| alignment_genome_start | 8 | 8 | 8 | 8 |
| alignment_genome_end | 3106840 | 3106840 | 3106840 | 3106840 |
| alignment_strand_start | 0 | 0 | 0 | 0 |
| alignment_strand_end | 3063309 | 3062755 | 3062755 | 3062755 |
| alignment_strand_coverage | 0.999999 | 0.999999 | 0.999999 | 0.999999 |
| alignment_length | 3159604 | 3156375 | 3156375 | 3156375 |
| alignment_num_aligned | 3010537 | 3013212 | 3013212 | 3013212 |
| alignment_num_correct | 2877221 | 2881045 | 2881045 | 2881045 |
| alignment_num_deletions | 96295 | 93620 | 93620 | 93620 |
| alignment_num_insertions | 52772 | 49543 | 49543 | 49543 |
| alignment_num_substitutions | 133316 | 132167 | 132167 | 132167 |

# The code below are legacy

In [24]:
# Assuming the .fasta file contain a single read

def read_fasta(file_path):
  sequence = ""
  with open(file_path, 'r') as file:
    for line in file:
      line = line.strip()
      if line.startswith(">"):
        continue  # Skip header
      sequence += line
  return sequence

ref_sequence = read_fasta("test/sample.fasta")
len(ref_sequence)

3106848

In [34]:
def read_fastq(file_path):
  with open(file_path, 'r') as file:
    header = file.readline().strip()
    seq = file.readline().strip()
    _ = file.readline()
    qual = file.readline().strip()

    read = {
      "sequence": seq,
      "quality": qual,
      "avg_qscore": float(header.split("qs:f:")[1].split()[0])
    }
  return read

simulated_read = read_fastq("test/sample1.fastq")
len(simulated_read['sequence']), len(simulated_read['quality']), simulated_read['avg_qscore']

(3055325, 3055325, 11.4038)

In [ ]:
import edlib

ref = "ACTGACTGACTGACTGACTGACTGACTGACTGACTGACTGACTGACTGACTGACTGACTGACTGACTGACTGACTGACTGACTGACTGACTGACTG"
simulated = "CTCGCTCGCTCGCTCGCTCGCTCGCTCGCTTCGCTCGCTCGCTCGCTCGCTCGCTCGCTCGCTCGCTCGCTCGCTTCGCTCCGCTCGAC"

# Align and compute identity
result = edlib.align(ref, simulated, task="path")
identity = (len(ref) - result["editDistance"]) / len(ref) * 100
print(f"Accuracy: {identity:.2f}%")

Accuracy: 48.96%


In [15]:
import edlib

def calculate_accuracy(ref_seq, simulated_seq):
    alignment = edlib.align(ref_seq, simulated_seq, task="path")
    matches = len(ref_seq) - alignment["editDistance"]
    identity = (matches / len(ref_seq)) * 100
    return identity

ref_seq = ref_sequences["a_tale_of_two_cities"]
simulated_seq = simulated_reads["93e5f2d6-797e-48a8-9357-174bfb09e38e"]["sequence"]
accuracy = calculate_accuracy(ref_seq, simulated_seq)
accuracy

40.396

In [17]:
import uuid

input_file = f"{data_dir}/sample.slow5"
output_file = f"{data_dir}/fixed_sample.slow5"

with open(input_file) as fin, open(output_file, "w") as fout:
  for line in fin:
    if line.startswith("#") or line.startswith("@"):
      fout.write(line)
    else:
      parts = line.strip().split('\t')
      parts[0] = str(uuid.uuid4())  # replace read_id with UUID
      fout.write('\t'.join(parts) + '\n')
